In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import MBartTokenizer, MBartForConditionalGeneration, DataCollatorForSeq2Seq, get_scheduler, MBart50TokenizerFast
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

df = pd.read_excel('/content/drive/MyDrive/CS549_Project/CS549_project.xlsx')
df = df[['Stock_Report', 'News']].copy()  # Warning fix
df["Stock_Report"] = "haberleştir: " + df["Stock_Report"]

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df.reset_index(drop=True))

model_name = "facebook/mbart-large-cc25"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
tokenizer.src_lang = "tr_XX"
tokenizer.tgt_lang = "tr_XX"
model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)

def tokenize_function(examples):
    model_inputs = tokenizer(examples['Stock_Report'], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['News'], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['Stock_Report', 'News'])
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=['Stock_Report', 'News'])

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(val_dataset, batch_size=4, collate_fn=data_collator)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=0.05)
num_epochs = 40

patience = 5
trigger_count = 0
best_val_loss = float("inf")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    print(f"\n🟢 Epoch {epoch+1}/{num_epochs}")

    for batch in tqdm(train_loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"📉 Training Loss     : {avg_loss:.4f}")

    # Validation
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f"🧪 Validation Loss  : {avg_val_loss:.4f}")

    # Early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        trigger_count = 0
    else:
        trigger_count += 1
        print(f"⚠️ Validation loss did not improve. Trigger count: {trigger_count}/{patience}")
        if trigger_count >= patience:
            print("⛔ Early stopping activated.")
            break

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBartTokenizer'. 
The class this function is called from is 'MBart50TokenizerFast'.


Map:   0%|          | 0/360 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/40 [00:00<?, ? examples/s]


🟢 Epoch 1/40


...

🧪 Validation Loss  : 4.1426
⚠️ Validation loss did not improve. Trigger count: 4/5

🟢 Epoch 8/40


Training: 100%|██████████| 90/90 [00:26<00:00,  3.39it/s]


📉 Training Loss     : 1.1632


Validating: 100%|██████████| 10/10 [00:00<00:00, 11.27it/s]

🧪 Validation Loss  : 2.3483
⚠️ Validation loss did not improve. Trigger count: 5/5
⛔ Early stopping activated.


In [ ]:
def generate_news(stock_report_text, model, tokenizer, device):
    prompt = "haberleştir: " + stock_report_text
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=256,
            num_beams=4,
            repetition_penalty=2.5,
            early_stopping=True
        )

    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return generated_text


In [ ]:
sample_stock_report = (
    "Şirketimizin 21.03.2025 tarihli özel durum açıklamasında, paylarının tamamına sahip olduğumuz bağlı ortaklığımız Marbaş Menkul Değerler AŞ'nin (Marbaş Menkul), 150.000.000 TL olan çıkarılmış sermayesinin 360.000.000 TL'ye çıkarılmasına, artırılan 210.000.000 TL sermayenin iç kaynaklardan karşılanmasına ilişkin olarak Marbaş Menkul tarafından Sermaye Piyasası Kurulu'na yapılan başvurunun olumlu karşılandığı duyurulmuştur."
)

generated_news = generate_news(sample_stock_report, model, tokenizer, device)
print("Üretilen Haber:")
print(generated_news)


Üretilen Haber:
Şirket, tamamına sahip olduğu bağlı ortaklığı Marbaş Menkul Değerler A.Ş.’nin sermaye artırımı sürecinde önemli bir adım attı. 150 milyon TL olan çıkarılmış sermaye, iç kaynaklardan karşılanmak üzere 360 milyon TL’ye yükseltildi. Sermaye Piyasası Kurulu da Marbaş tarafından yapılan başvuruyu olumlu karşılayarak artışı onayladı. Bu adım, Marbaş’ın uzun vadeli büyüme stratejisine işaret ediyor gibi görünüyor. Açıkçası, iç kaynakları güçlü bir yapıya kavuşması ve Marbaş’ın güçlü özkaynak yapısı korunması açısından olumlu bir sinyal


In [ ]:
sample_stock_report = ("Sermayesine %49 oranında ortaklığımızın bulunduğu iştiraklerimiz Sour Turizm A.Ş. ve Çağrankaya Turizm A.Ş.'nin sermayelerinin 250.000.000 TL'den 400.000.000 TL'ye çıkarılmasına karar verilmiştir.")
generated_news = generate_news(sample_stock_report, model, tokenizer, device)
print("Üretilen Haber:")
print(generated_news)

Üretilen Haber:
Şirket, %49 oranında bağlı ortaklığı olan SOUR Turizm A.Ş. ve Çağrankaya Turizm A.Ş.’nin sermayesini 25 milyon TL’den 40 milyon TL’ye çıkarma kararı aldı. Bu adım, sadece turizm değil, aynı zamanda sosyal sorumlulukları açısından da büyük bir adım olarak görülüyor. Açıkçası, söz konusu karar, şirketin uzun vadeli büyüme stratejisine işaret ediyor gibi görünüyor.
